In [1]:
import os
import numpy as np
from PIL import Image
import skimage.color as sc


def get_all_img_paths_gen(path_root, min_h=0, min_w=0):
    for (dirpath, dirnames, filenames) in os.walk(path_root):
        filenames = [f for f in filenames if not f[0] == '.']
        dirnames[:] = [d for d in dirnames if not d[0] == '.']

        for file in filenames:
            if (file.endswith(tuple(['.bmp', '.jpg', '.png']))):
                path = os.path.join(dirpath, file)
                if min_h == 0 and min_w == 0:
                    yield path
                else:
                    img = Image.open(path)
                    if(img.mode != 'RGB'):
                        continue
                    img_size = img.size
                    h = img_size[1]
                    w = img_size[0]
                    if(min_h <= h and min_w <= w):
                        yield path
                    
def get_all_img_paths(path_root, min_h=0, min_w=0):
    paths = []
    for path in get_all_img_paths_gen(path_root, min_h, min_w):
        paths.append(path)
    return sorted(paths)
                    
def psnr(img1, img2, ycbcr=False, shave=0):
    if ycbcr:
        a = np.float32(img1)
        b = np.float32(img2)
        a = sc.rgb2ycbcr(a / 255)[:, :, 0]
        b = sc.rgb2ycbcr(b / 255)[:, :, 0]
    else:
        a = np.array(img1).astype(np.float32)
        b = np.array(img2).astype(np.float32)
        
    if shave:
        a = a[shave:-shave, shave:-shave]
        b = b[shave:-shave, shave:-shave]
    
    mse = np.mean((a - b) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return np.minimum(100.0, 20 * np.math.log10(PIXEL_MAX) - 10 * np.math.log10(mse))


def resize_img(img_np, height, width, resample=Image.BICUBIC):
    img = Image.fromarray(img_np)
    resized = img.resize((width, height), resample=resample)
    return np.array(resized)

In [2]:
dbs = []
paths = get_all_img_paths('./datasets/SR_testing_datasets/Set5')

for path in paths:
    gt = Image.open(path)
    gt = np.asarray(gt)
    h, w, c = gt.shape
    bicubic = resize_img(gt, h//2, w//2)
    bicubic = resize_img(bicubic, h, w)
    
    db = psnr(gt, bicubic, ycbcr=True)
    dbs.append(db)
    print (path, dbs[-1])

print (np.mean(dbs))

./datasets/SR_testing_datasets/Set5/baby.png 37.109061386427015
./datasets/SR_testing_datasets/Set5/bird.png 36.730937342001866
./datasets/SR_testing_datasets/Set5/butterfly.png 27.46215440136178
./datasets/SR_testing_datasets/Set5/head.png 34.90124739013607
./datasets/SR_testing_datasets/Set5/woman.png 32.159485376308346
33.672577179247014
